In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import time
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [2]:
datos = pd.read_csv("FOTOSINTESIS.csv", delimiter=";")

In [3]:
datos = datos.drop(['Ano','Cod','Bloque'], axis=1)

In [4]:
datos = datos.drop(['Altura planta','Area de copa (A) =m2','Asimilacion de CO2','Uso eficiente del agua','Transpiracion','Conductividad','K','MO','Conductividad electrica','Lluvias acumuladas (mms)','Humedad'], axis=1)

In [5]:
datos = datos.fillna(datos.mean())

In [6]:
cols_to_norm = ['Diametro tronco (cm)','A.F no destructiva m2','P','Radiacion','Temperatura',' Numero de dias con lluvia','Total']
datos[cols_to_norm] = datos[cols_to_norm].apply(lambda X: (X - X.min()) / (X.max() - X.min()))

In [7]:
datos.head()

,Diametro tronco (cm),A.F no destructiva m2,P,Radiacion,Temperatura,Numero de dias con lluvia,Total
0,0.315053,0.239180,0.063124,0.404994,0.980408,0.666667,0.081196
1,0.310385,0.153458,0.063124,0.404994,0.980408,0.666667,0.042171
2,0.345391,0.303673,0.063124,0.404994,0.980408,0.666667,0.018566
3,0.190198,0.033923,0.063124,0.404994,0.980408,0.666667,0.153303
4,0.353559,0.186011,0.063124,0.404994,0.980408,0.666667,0.046181


In [8]:
# Especificar X y Y
X = datos.values[:,:-1]  
Y = datos.values[:,-1]

In [9]:
# hacer validación cruzada
#kf = KFold(n_splits=5,random_state=0, shuffle=True)
#kf.get_n_splits(X)

In [10]:
# para hacer holdout
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42, shuffle=True)

In [11]:
mlp = MLPRegressor(solver='adam', hidden_layer_sizes=10, max_iter=200, activation='relu',random_state=1, learning_rate_init=0.025,batch_size='auto', momentum=0.09)

In [12]:
#d = [(e,) for e in range (1,3)]
#d = [ (e, ) for e in range (1,11)]
c = [(a,b) for a in range(4,11) for b in range(1,6)]

#print(d)
print(c)
#print(len(d))
print(len(c))

[(4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (10, 1), (10, 2), (10, 3), (10, 4), (10, 5)]
35


In [13]:
#f = c+d
#print(f)
#print(len(f))

In [14]:
grid_param = {  
    'solver': ['lbfgs', 'sgd', 'adam'],
    'activation': ['relu', 'logistic', 'tanh'],
    'hidden_layer_sizes': c,
    'momentum': [0.09, 0.9],
    'learning_rate_init': [0.025, 0.05]
}

In [15]:
gd_sr = GridSearchCV(estimator=mlp,  
                     param_grid=grid_param,
                     scoring='neg_mean_absolute_error',
                     cv=5,
                     n_jobs=-1)

In [16]:
start = time.time()

gd_sr.fit(X_train, y_train)

end = time.time()
print(end - start)

88.39873838424683


In [17]:
best_parameters = gd_sr.best_params_  
print(best_parameters)  

{'activation': 'relu', 'hidden_layer_sizes': (6, 3), 'learning_rate_init': 0.025, 'momentum': 0.09, 'solver': 'lbfgs'}


In [18]:
gd_sr.best_score_

-0.10315776089043763